# Converting a trained model to tflite
https://www.tensorflow.org/lite/microcontrollers/build_convert#model_conversion

# Convert model to tflite

In [1]:
import tensorflow as tf
from tensorflow.io import gfile
import numpy as np
import random

2022-08-09 22:47:08.457469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-09 22:47:08.457589: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
MODEL_PATH = 'trained.model'
DATA_DIR = '/home/Radhi/Desktop/TinyML_Book/voice-controlled-robot/model/speech_data_classes_npy'
QUANTIZED_MODEL_PATH = 'converted_model.tflite'

In [8]:
def get_spectr_files(source, word):
    return gfile.glob(source + '/'+word+'/*.npy')

In [10]:
classes = [
    'yes',
    'off',
    'left',
    'right',
    '_invalid',
]

test_spectr = []
for word in classes:
  for file in random.sample(get_spectr_files(DATA_DIR, word), 200):
    test_spectr.append(np.load(file))
test_ds = np.array(test_spectr)


In [11]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_PATH)

def representative_data_generator():
    img = tf.data.Dataset.from_tensor_slices(test_ds).batch(1)
    for i in img.take(100):
        yield [i]

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.representative_dataset = representative_data_generator

tflite_quant_model = converter.convert()

with open(QUANTIZED_MODEL_PATH, "wb") as f:
    f.write(tflite_quant_model)

2022-08-09 22:50:18.801115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-09 22:50:18.808022: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-09 22:50:18.930032: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (RS): /proc/driver/nvidia/version does not exist
2022-08-09 22:50:19.080525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-09 22:50:22.512684: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_form

# To convert to C array

In [12]:
!xxd -i converted_model.tflite > model_data.cc